# ST-RRED - Computes Spatio-Temporal Reduced Reference Entropic Differencing

#### Source: http://www.scikit-video.org/stable/measure.html

Use `skvideo.measure.strred` to measure the perceptually quality difference between two videos, providing a slightly reduced reference score along with a critically reduced reference score (1 number comparison between videos) (sciki-video,2021). See [1](https://ieeexplore.ieee.org/document/6279462).

 - [1] R. Soundararajan and A. C. Bovik, "Video Quality Assessment by Reduced Reference Spatio-temporal Entropic Differencing," IEEE Transactions on Circuits and Systems for Video Technology, April 2013.

## Example:

In this example, video inputs are compared over frame differences, with quality determined by differences in the entropy per subband.

The distorted video is upscaled to match the resolution of the reference video:

In [ ]:
!ffmpeg -hide_banner -y -i videos/dist/x264/bbb_640x360_CRF40.mp4 -vf scale=1920x1080 -c:v libx264 -crf 0 /tmp/dist.mp4

<div class="alert alert-block alert-info">
    <b>Important:</b> To run this code, switch to the <b><i>Scikit-Video</i></b> kernel. Button in the upper right corner.</div>

In [ ]:
import skvideo.io
import skvideo.datasets
import skvideo.utils
import numpy as np
import skvideo.measure
import pandas as pd
import os


dir='results/strred'
if not os.path.exists(dir):
    os.makedirs(dir)

# Distorted and reference videos (adapt to your needs)
path_distorted='/tmp/dist.mp4'
path_reference='videos/ref/bbb_1920x1080.mp4'

v_distorted= skvideo.io.vread(path_distorted, outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
v_reference = skvideo.io.vread(path_reference, outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
      
print 'Running, wait until score is obtained (this can take a while...)'
    
str_data=skvideo.measure.strred(v_reference, v_distorted)
str_lis=[]

for a in str_data[0]:
    str_lis.append([a[0],a[1],a[2],a[3]])

df=pd.DataFrame(str_lis, columns=['rreds','rredt','rredssn','rredtsn'])

strred=df.rreds.mean()*df.rredt.mean()
strredsn=df.rredssn.mean()*df.rredtsn.mean()


print 'ST-RRED score: \n Full-reference: {1}  \n Reduced-reference: {0}'.format(strred, strredsn)

#Save information in .json
df.to_json('results/strred/strred.json', index=True)

## Graphical representation.

<div class="alert alert-block alert-warning">
<b>Warning:</b> To show the graphical representation, switch to the <b><i>Python 3</i></b> kernel. Button in the upper right corner.
</div>

In [ ]:
import pandas as pd 
import re
import plotly.graph_objects as go
import json

# File with metric data (adapt to your needs)
path_file='results/strred/strred.json'

df=pd.read_json(path_file)
df=df.sort_index()

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df.rreds*df.rredt,
                    mode='lines+markers',
                    name='STRRED',
                         line=dict(color='#3E4680'))) 

fig.add_trace(go.Scatter(x=df.index, y=df.rredssn*df.rredtsn,
                    mode='lines+markers',
                    name='STRREDSN',
                         line=dict(color='#C56E90'))) 

fig.update_layout(
    title={'text':'STRRED by frame difference',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True,
    template='simple_white',
    xaxis_title='Frame difference')

fig.show()